In [7]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point


In [8]:
# load GeoJSON file（570 polygons）
polygons = gpd.read_file('/Users/koko.lv/Documents/GitHub/colloquium_2_2024/Data/BK/brooklyn_grid_300m_within_polygon.geojson')

# load muiltiple csv files (with lat,lon,attributes)
csv_files = ['/Users/koko.lv/Documents/GitHub/colloquium_2_2024/Data/BK/BK_RentPricing_zillow_2024_updated_geolocation.csv', 
             '/Users/koko.lv/Documents/GitHub/colloquium_2_2024/Data/BK/googlemaps_nearby_musicVenue_20collection.csv', 
             '/Users/koko.lv/Documents/GitHub/colloquium_2_2024/Data/BK/NumFloor_updated.csv']
dataframes = [pd.read_csv(file) for file in csv_files]

# integrateall CSV to a single DataFrame
points_data = pd.concat(dataframes, ignore_index=True)

# rename latitude 和 longitude
points_data = points_data.rename(columns={"latitude": "lat", "longitude": "lon"})


In [9]:
from shapely.geometry import Point

# tranform latitude and longitude to geometry (Point) 
geometry = [Point(xy) for xy in zip(points_data['lon'], points_data['lat'])]
points_gdf = gpd.GeoDataFrame(points_data, geometry=geometry)

# make sure data and polygon share the same CRS 
points_gdf.crs = polygons.crs


In [10]:
# spatial join
joined = gpd.sjoin(points_gdf, polygons, how="inner", predicate="within")

# result?
print(joined.head())

# joined 包含點數據的屬性和多邊形的屬性，現在可以進一步處理。


   ID_left neighbourhood                     address        lat        lon  \
2      3.0    Greenpoint    527 Leonard St, Brooklyn  40.722986 -73.949610   
3      4.0    Greenpoint    109 Russell St, Brooklyn  40.724278 -73.944529   
4      5.0    Greenpoint      16 Dupont St, Brooklyn  40.735521 -73.957986   
5      6.0    Greenpoint  87 Commercial St, Brooklyn  40.737469 -73.956543   
6      7.0    Greenpoint    536 Graham Ave, Brooklyn  40.721274 -73.946725   

     type   price  zipcode  Index Name Type Address  BuildingType  NumFloors  \
2  Studio  2450.0  11222.0    NaN  NaN  NaN     NaN           NaN        NaN   
3      1b  3450.0  11222.0    NaN  NaN  NaN     NaN           NaN        NaN   
4      1b  4395.0  11222.0    NaN  NaN  NaN     NaN           NaN        NaN   
5      1b  4475.0  11222.0    NaN  NaN  NaN     NaN           NaN        NaN   
6      1b  2600.0  11222.0    NaN  NaN  NaN     NaN           NaN        NaN   

   LandUse                    geometry  index_righ

In [14]:
print(joined.columns)  # 查看 'joined' 中的列名
print(polygons.columns)  # 查看 'polygons' 中的列名


Index(['ID_left', 'neighbourhood', 'address', 'lat', 'lon', 'type', 'price',
       'zipcode', 'Index', 'Name', 'Type', 'Address', 'BuildingType',
       'NumFloors', 'LandUse', 'geometry', 'index_right', 'ID_right'],
      dtype='object')
Index(['ID', 'geometry'], dtype='object')


In [16]:
print(joined.columns)

Index(['ID_left', 'neighbourhood', 'address', 'lat', 'lon', 'type', 'price',
       'zipcode', 'Index', 'Name', 'Type', 'Address', 'BuildingType',
       'NumFloors', 'LandUse', 'geometry', 'index_right', 'ID_right'],
      dtype='object')


In [37]:
print(polygons.columns)  # 查看 polygons 中的列名
print(aggregated_data.columns)  # 查看 aggregated_data 中的列名

Index(['polygon_id', 'geometry'], dtype='object')
Index(['ID_left', 'price', 'NumFloors'], dtype='object')


In [38]:
polygons = polygons.rename(columns={'polygon_id': 'ID_left'})


In [39]:
print(polygons.head())

   ID_left                                           geometry
0        1  POLYGON ((-73.98524 40.67607, -73.98475 40.676...
1        2  POLYGON ((-73.98524 40.67607, -73.98524 40.678...
2        3  POLYGON ((-73.98254 40.68147, -73.98055 40.681...
3        4  POLYGON ((-73.96364 40.68147, -73.96364 40.681...
4        5  POLYGON ((-73.96094 40.68147, -73.95824 40.681...


In [40]:
print(polygons.columns)  # 查看 polygons 中的列名
print(aggregated_data.columns)  # 查看 aggregated_data 中的列名


Index(['ID_left', 'geometry'], dtype='object')
Index(['ID_left', 'price', 'NumFloors'], dtype='object')


In [41]:
# 假設多邊形 ID 字段為 'polygon_id'，CSV 中的數值屬性為 'value'
# 按多邊形進行聚合，這裡以平均值為例
aggregated_data = joined.groupby('ID_left').agg({
    'price': 'mean',  # 可以是 'sum', 'max', 'min', 等
    'NumFloors': 'sum'  # 其他屬性
}).reset_index()

# 將聚合結果與多邊形結合
polygons = polygons.merge(aggregated_data, on='ID_left', how='left')


In [42]:
polygons.to_file('/Users/koko.lv/Documents/GitHub/colloquium_2_2024/Data/BK/spatialJoin_outcome.geojson', driver='GeoJSON')

print("整合完成，結果已保存為 'output.geojson'")


整合完成，結果已保存為 'output.geojson'


In [48]:
import geopandas as gpd

file_path = '/Users/koko.lv/Documents/GitHub/colloquium_2_2024/Data/BK/spatialJoin_outcome.geojson'
gdf = gpd.read_file(file_path)
print(gdf)


     ID_left   price  NumFloors  \
0          1     NaN        NaN   
1          2     NaN        1.0   
2          3  2450.0        0.0   
3          4  3450.0        0.0   
4          5  4395.0        0.0   
..       ...     ...        ...   
565      566     NaN        0.0   
566      567     NaN        0.0   
567      568     NaN        3.0   
568      569     NaN        3.0   
569      570  3400.0        3.0   

                                              geometry  
0    POLYGON ((-73.98524 40.67607, -73.98475 40.676...  
1    POLYGON ((-73.98524 40.67607, -73.98524 40.678...  
2    POLYGON ((-73.98254 40.68147, -73.98055 40.681...  
3    POLYGON ((-73.96364 40.68147, -73.96364 40.681...  
4    POLYGON ((-73.96094 40.68147, -73.95824 40.681...  
..                                                 ...  
565  POLYGON ((-73.94280 40.73277, -73.94276 40.732...  
566  POLYGON ((-73.92314 40.71117, -73.92314 40.712...  
567  POLYGON ((-73.92584 40.71387, -73.92584 40.714...  
568  POLY